In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support

In [2]:
#read the pickle files
not_aug_df=pd.read_pickle('EMOVO_dataset/not_aug_df.pkl')
semi_aug_df=pd.read_pickle('EMOVO_dataset/semi_aug_df.pkl')
aug_df=pd.read_pickle('EMOVO_dataset/aug_df.pkl')

In [3]:
# test_df = not_aug_df.loc[not_aug_df["actor"] == "f1"]
# not_aug_df = not_aug_df.drop(test_df.index)

# Y_test = test_df["label_id"]
# X_mfccs_k_test = np.array(test_df.iloc[:, 3:29])
# X_mfccs_k_test=np.array(X_mfccs_k_test.tolist())
# X_mfccs_k_test=scaler.fit_transform(X_mfccs_k_test.reshape(-1, X_mfccs_k_test.shape[-1])).reshape(X_mfccs_k_test.shape)
# Y_mfccs_k_test=test_df['label']
# X_logmel_test = np.array(test_df.loc[:, ['logmel' in i for i in test_df.columns]])
# X_logmel_test=np.array(X_logmel_test.tolist())
# X_test = np.reshape(X_logmel_test, (X_logmel_test.shape[0],X_logmel_test.shape[1],X_logmel_test.shape[2],1))
# X_logmel_test = scaler.fit_transform(X_logmel_test.reshape(-1, X_logmel_test.shape[-1])).reshape(X_logmel_test.shape)


In [4]:
len(not_aug_df)

588

In [5]:
#creation of the array containg the emotions ordered by their encoding
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
sorted_labels=[]
label_encoder.fit(not_aug_df['label'])
name_mapping = dict(zip( label_encoder.transform(label_encoder.classes_),label_encoder.classes_))
for i in range(7):
  sorted_labels.append(name_mapping[i])
sorted_labels

['anger', 'disgust', 'fear', 'joy', 'neutrality', 'sadness', 'surprise']

In [6]:
scaler = StandardScaler()
#extraction of the mfccs from the datasets - not aug
X_mfccs_k = np.array(not_aug_df.iloc[:, 3:29])
X_mfccs_k=np.array(X_mfccs_k.tolist())
X_mfccs_k=scaler.fit_transform(X_mfccs_k.reshape(-1, X_mfccs_k.shape[-1])).reshape(X_mfccs_k.shape)
Y_mfccs_k=not_aug_df['label']


#extraction of the log mel specrogram from the datasets - not aug
X_logmel_k = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
X_logmel_k=np.array(X_logmel_k.tolist())
X_logmel_k=scaler.fit_transform(X_logmel_k.reshape(-1, X_logmel_k.shape[-1])).reshape(X_logmel_k.shape)
Y_logmel_k=not_aug_df['label']

#reshape the data from 3D to 2D - not aug
X_mfccs_k=X_mfccs_k.reshape(X_mfccs_k.shape[0],X_mfccs_k.shape[1]*X_mfccs_k.shape[2])
X_logmel_k=X_logmel_k.reshape(X_logmel_k.shape[0],X_logmel_k.shape[1]*X_logmel_k.shape[2])

In [7]:
#extraction of labels_id from datasets
Y_not_aug=not_aug_df['label_id']
Y_semi_aug=semi_aug_df['label_id']
Y_aug=aug_df['label_id']


#take the log mel spectrogram from the datasets
X_logmel = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
X_logmel=np.array(X_logmel.tolist())
X_logmel_semi_aug = np.array(semi_aug_df.loc[:, ['logmel' in i for i in semi_aug_df.columns]])
X_logmel_semi_aug=np.array(X_logmel_semi_aug.tolist())
X_logmel_aug = np.array(aug_df.loc[:, ['logmel' in i for i in aug_df.columns]])
X_logmel_aug=np.array(X_logmel_aug.tolist())


#take the mfccs & deltas from the datasets
X_mfccs = np.array(not_aug_df.iloc[:, 3:29]) 
X_mfccs=np.array(X_mfccs.tolist())
X_mfccs_semi_aug = np.array(semi_aug_df.iloc[:, 3:29])
X_mfccs_semi_aug=np.array(X_mfccs_semi_aug.tolist())
X_mfccs_aug = np.array(aug_df.iloc[:, 3:29])
X_mfccs_aug=np.array(X_mfccs_aug.tolist())

In [8]:
X_logmel.shape

(588, 60, 130)

In [9]:
#add one dimension to data to make them suitable foR CNN
X_logmel = np.reshape(X_logmel, (X_logmel.shape[0],X_logmel.shape[1],X_logmel.shape[2],1))
X_logmel_semi_aug= np.reshape(X_logmel_semi_aug, (X_logmel_semi_aug.shape[0],X_logmel_semi_aug.shape[1],X_logmel_semi_aug.shape[2],1))
X_logmel_aug= np.reshape(X_logmel_aug, (X_logmel_aug.shape[0],X_logmel_aug.shape[1],X_logmel_aug.shape[2],1))

In [10]:
X_logmel.shape

(588, 60, 130, 1)

In [11]:
#CNN creation
def create_cnn(input_data):
  modelcnn = keras.Sequential()

  modelcnn.add(keras.layers.Conv2D(32, (2, 6), activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7), strides=(2,2),padding='same'))

  modelcnn.add(keras.layers.Conv2D(64, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2,7),strides=(2,2), padding='same'))

  modelcnn.add(keras.layers.Conv2D(128, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7),strides=(2,2), padding='same'))

  modelcnn.add(keras.layers.Conv2D(256, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7),strides=(2,2), padding='same'))

 
  modelcnn.add(keras.layers.GlobalAveragePooling2D())
  modelcnn.add(keras.layers.Dense(128, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))

  modelcnn.add(keras.layers.Dense(64, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))

  modelcnn.add(keras.layers.Dense(7, activation='softmax'))
  
  return modelcnn

In [12]:
def compile_model():
    input_data=(X_logmel.shape[1:])
    modelcnn=create_cnn(input_data)
    modelcnn.compile(
    optimizer = "Adam", 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
    )
    return modelcnn
def train_val_test(X,Y):
    X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=22)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=22)
    return X_train,X_val,X_test,y_train,y_val,y_test

def get_accuracy(X_test,y_test, modelcnn):
    predicted = modelcnn.predict(X_test)

    preds = []
    for prediction in predicted:
        preds.append(np.argmax(prediction))

    print(accuracy_score(y_test,preds))


In [13]:
from datetime import datetime  
name = datetime.now().strftime("ser_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]
modelcnn = compile_model()


X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel,Y_not_aug)



history = modelcnn.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=256,
                       epochs=1000,
                       callbacks=callbacks)

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 796ms/step - accuracy: 0.1459 - loss: 2.5041
Epoch 1: val_loss improved from inf to 2.45141, saving model to ser_11_09_2024_13_01_32.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.1438 - loss: 2.5051 - val_accuracy: 0.1132 - val_loss: 2.4514
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 812ms/step - accuracy: 0.1691 - loss: 2.4516
Epoch 2: val_loss improved from 2.45141 to 2.43952, saving model to ser_11_09_2024_13_01_32.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.1734 - loss: 2.4497 - val_accuracy: 0.1792 - val_loss: 2.4395
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 834ms/step - accuracy: 0.2059 - loss: 2.4041
Epoch 3: val_loss improved from 2.43952 to 2.33349, saving model to ser_11_09_2024_13_01_32.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.2043 - loss: 2.4029 - val_accuracy: 0.2453 - val_loss: 2.3335
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 835ms/step - accuracy: 0.2045 - loss: 2.3541
Epoch 4: val_loss improved fro

In [14]:
get_accuracy(X_test,y_test,modelcnn)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 242ms/step
0.4067796610169492


In [15]:
modelcnn = compile_model()

X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_semi_aug,Y_semi_aug)

In [16]:
from datetime import datetime  
name = datetime.now().strftime("ser_semiaug_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]

history_semi = modelcnn.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=256,
                       epochs=1000,
                       callbacks=callbacks)

Epoch 1/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1597 - loss: 2.4902
Epoch 1: val_loss improved from inf to 2.42887, saving model to ser_semiaug_11_09_2024_13_04_06.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.1606 - loss: 2.4880 - val_accuracy: 0.1761 - val_loss: 2.4289
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2037 - loss: 2.3890
Epoch 2: val_loss improved from 2.42887 to 2.31790, saving model to ser_semiaug_11_09_2024_13_04_06.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.2043 - loss: 2.3857 - val_accuracy: 0.2201 - val_loss: 2.3179
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2537 - loss: 2.2844
Epoch 3: val_loss improved from 2.31790 to 2.24640, saving model to ser_semiaug_11_09_2024_13_04_06.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.2528 - loss: 2.2825 - val_accuracy: 0.2642 - val_loss: 2.2464
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2451 - loss: 2.2097
Epoch 4: val_los

KeyboardInterrupt: 

In [188]:
get_accuracy(X_test,y_test,modelcnn)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
0.864406779661017


In [189]:
modelcnn = compile_model()

X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_aug,Y_aug)

In [190]:
from datetime import datetime  
name = datetime.now().strftime("ser_aug_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


history_semi = modelcnn.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=256,
                       epochs=1000,
                       callbacks=callbacks)

Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.1509 - loss: 2.4782
Epoch 1: val_loss improved from inf to 2.36049, saving model to ser_aug_10_09_2024_13_30_46.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 547ms/step - accuracy: 0.1542 - loss: 2.4743 - val_accuracy: 0.2689 - val_loss: 2.3605
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step - accuracy: 0.2612 - loss: 2.3146
Epoch 2: val_loss improved from 2.36049 to 2.27523, saving model to ser_aug_10_09_2024_13_30_46.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 561ms/step - accuracy: 0.2603 - loss: 2.3135 - val_accuracy: 0.2642 - val_loss: 2.2752
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 0.2341 - loss: 2.2389
Epoch 3: val_loss improved from 2.27523 to 2.21132, saving model to ser_aug_10_09_2024_13_30_46.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 554ms/step - accuracy: 0.2339 - loss: 2.2374 - val_accuracy: 0.2594 - val_loss: 2.2113
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step - accuracy: 0.2637 - loss: 2.1671
Epoch 4: 

In [191]:
get_accuracy(X_test,y_test,modelcnn)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.9067796610169492


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from keras import layers, models


In [13]:
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel,Y_not_aug)
X_train.shape

(423, 60, 130, 1)

In [14]:
def get_model():
    inputs = layers.Input(shape=(60,130))
    encoder = layers.LSTM(128)(inputs)
    drop = layers.Dropout(0.3)(encoder)
    hidden = layers.Dense(32, activation='relu')(drop)
    outputs = layers.Dense(7, activation='softmax')(hidden)
    
    model = models.Model(inputs, outputs)
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=["accuracy"])

    return model

In [15]:
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel.squeeze(3),Y_not_aug)
LSTM_model = get_model()
LSTM_model.summary()


from datetime import datetime  
name = datetime.now().strftime("ser_lstm_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


LSTM_history = LSTM_model.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=32,
                       epochs=1000,
                       verbose=1,
                       callbacks=callbacks)


print(f"Loss : {LSTM_model.evaluate(X_test,y_test)[0]}, Accuracy : {LSTM_model.evaluate(X_test,y_test)[1]}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 60, 130)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       132,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,967 (535.03 KB)

 Trainable params: 136,967 (535.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1224 - loss: 1.9938
Epoch 1: val_loss improved from inf to 1.98349, saving model to ser_lstm_11_09_2024_11_22_39.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.1247 - loss: 1.9909 - val_accuracy: 0.1604 - val_loss: 1.9835
Epoch 2/1000
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.2875 - loss: 1.8174
Epoch 2: val_loss improved from 1.98349 to 1.98052, saving model to ser_lstm_11_09_2024_11_22_39.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.2848 - loss: 1.8211 - val_accuracy: 0.1415 - val_loss: 1.9805
Epoch 3/1000
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.3129 - loss: 1.7464
Epoch 3: val_loss did not improve from 1.98052
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.3163 - loss: 1.7450 - val_accuracy: 0.1415 - val_loss: 2.0203
Epoch 4/1000
12/14 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4338 - loss: 1.6119
Epoch 4: val_loss did not improve from 1.98052
14/14 ━━

In [16]:
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_semi_aug.squeeze(3),Y_semi_aug)
LSTM_model = get_model()
LSTM_model.summary()


from datetime import datetime  
name = datetime.now().strftime("ser_lstm_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


LSTM_history = LSTM_model.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=32,
                       epochs=1000,
                       verbose=1,
                       callbacks=callbacks)


print(f"Loss : {LSTM_model.evaluate(X_test,y_test)[0]}, Accuracy : {LSTM_model.evaluate(X_test,y_test)[1]}")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 60, 130)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       132,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,967 (535.03 KB)

 Trainable params: 136,967 (535.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.1587 - loss: 1.9887
Epoch 1: val_loss improved from inf to 1.83600, saving model to ser_lstm_11_09_2024_11_22_52.keras
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.1605 - loss: 1.9855 - val_accuracy: 0.2453 - val_loss: 1.8360
Epoch 2/1000
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.2819 - loss: 1.7779
Epoch 2: val_loss improved from 1.83600 to 1.76401, saving model to ser_lstm_11_09_2024_11_22_52.keras
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.2826 - loss: 1.7774 - val_accuracy: 0.2767 - val_loss: 1.7640
Epoch 3/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3741 - loss: 1.6249
Epoch 3: val_loss improved from 1.76401 to 1.61461, saving model to ser_lstm_11_09_2024_11_22_52.keras
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.3734 - loss: 1.6253 - val_accuracy: 0.3962 - val_loss: 1.6146
Epoch 4/1000
39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4221 - loss: 1.5032

In [17]:
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_aug.squeeze(3),Y_aug)
LSTM_model = get_model()
LSTM_model.summary()


from datetime import datetime  
name = datetime.now().strftime("ser_lstm_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


LSTM_history = LSTM_model.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=32,
                       epochs=1000,
                       verbose=1,
                       callbacks=callbacks)


print(f"Loss : {LSTM_model.evaluate(X_test,y_test)[0]}, Accuracy : {LSTM_model.evaluate(X_test,y_test)[1]}")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 60, 130)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       132,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,967 (535.03 KB)

 Trainable params: 136,967 (535.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
52/53 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1691 - loss: 1.9619
Epoch 1: val_loss improved from inf to 1.82668, saving model to ser_lstm_11_09_2024_11_24_36.keras
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.1699 - loss: 1.9606 - val_accuracy: 0.2901 - val_loss: 1.8267
Epoch 2/1000
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2912 - loss: 1.8070
Epoch 2: val_loss improved from 1.82668 to 1.69720, saving model to ser_lstm_11_09_2024_11_24_36.keras
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.2913 - loss: 1.8065 - val_accuracy: 0.3443 - val_loss: 1.6972
Epoch 3/1000
51/53 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.3720 - loss: 1.6195
Epoch 3: val_loss improved from 1.69720 to 1.54907, saving model to ser_lstm_11_09_2024_11_24_36.keras
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.3722 - loss: 1.6184 - val_accuracy: 0.4198 - val_loss: 1.5491
Epoch 4/1000
51/53 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4454 - loss: 1.4739

In [18]:
from sklearn.svm import SVC
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel.squeeze(3),Y_not_aug)
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)


SVC_model = SVC(kernel = 'rbf', gamma = 'auto', probability = True, verbose=True)


SVC_history = SVC_model.fit(X_train, y_train)

print(f"Accuracy : {accuracy_score(SVC_model.predict(X_test),np.array(y_test))}")

[LibSVM]Accuracy : 0.3220338983050847


In [19]:
from sklearn.svm import SVC
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_semi_aug.squeeze(3),Y_semi_aug)
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)


SVC_model = SVC(kernel = 'rbf', gamma = 'auto', probability = True, verbose=True)


SVC_history = SVC_model.fit(X_train, y_train)

print(f"Accuracy : {accuracy_score(SVC_model.predict(X_test),np.array(y_test))}")

[LibSVM]Accuracy : 0.6384180790960452


In [20]:
from sklearn.svm import SVC
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_aug.squeeze(3),Y_aug)
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)


SVC_model = SVC(kernel = 'rbf', gamma = 'auto', probability = True, verbose=True)


SVC_history = SVC_model.fit(X_train, y_train)

print(f"Accuracy : {accuracy_score(SVC_model.predict(X_test),np.array(y_test))}")

[LibSVM]Accuracy : 0.7966101694915254
